In [21]:
import re

# Exercise 7.1:
# Write a Spark job to count the occurrences of each word in a text file. Document that it works with a small example.

my_text = sc.textFile("../tests/meditations.txt")
split_words = my_text.flatMap(lambda line: re.findall('\w+', line, 0))
map_count = split_words.map(lambda word: (word, 1))
total_count = map_count.reduceByKey(lambda a, b: a + b) 
total_count.collect()

[(u'birth', 1),
 (u'hate', 1),
 (u'arrogant', 1),
 (u'We', 1),
 (u'implicate', 1),
 (u'but', 1),
 (u'Meditations', 1),
 (u'to', 3),
 (u'wrongdoer', 1),
 (u'has', 1),
 (u'Nor', 1),
 (u'And', 1),
 (u'them', 1),
 (u'good', 2),
 (u'they', 1),
 (u'hands', 1),
 (u'not', 1),
 (u'yourself', 1),
 (u'him', 2),
 (u'like', 3),
 (u'dishonest', 1),
 (u'morning', 1),
 (u'these', 1),
 (u't', 1),
 (u'each', 1),
 (u'obstruct', 1),
 (u'upper', 1),
 (u'because', 1),
 (u'From', 1),
 (u'deal', 1),
 (u'people', 1),
 (u'back', 1),
 (u'related', 1),
 (u'relative', 1),
 (u'born', 1),
 (u'are', 2),
 (u'surly', 1),
 (u'divine', 1),
 (u'unnatural', 1),
 (u'obstructions', 1),
 (u'be', 1),
 (u'nature', 1),
 (u'on', 1),
 (u'of', 6),
 (u'recognized', 1),
 (u'turn', 1),
 (u'or', 2),
 (u'own', 1),
 (u'No', 1),
 (u'feel', 2),
 (u'possessing', 1),
 (u'When', 1),
 (u'one', 1),
 (u'feet', 1),
 (u'your', 1),
 (u'rows', 1),
 (u'from', 1),
 (u'angry', 1),
 (u'two', 1),
 (u'But', 1),
 (u'Aurelius', 1),
 (u'tell', 2),
 (u'today'

In [110]:
# Exercise 7.2:
# Write a Spark job that determines if a graph has an Euler tour (all vertices have even degree) where you can assume that the graph you get is connected. 
# This file: 
# https://www.dropbox.com/s/usdi0wpsqm3jb7f/eulerGraphs.txt?dl=0 
# has 5 graphs – for each graph, the first line tells the number of nodes N and the number of edges E. 
# The next E lines tells which two nodes are connected by an edge. 
# Two nodes can be connected by multiple edges.
# It is fine if you split the file into 5 different files. You do not need to keep the node and edge counts in the top of the file.
# Document that it works using a small example.

graphs = ["../tests/euler/1.txt", "../tests/euler/2.txt", "../tests/euler/3.txt",
         "../tests/euler/4.txt", "../tests/euler/5.txt"]

def map_func(x):
    if x[1] % 2 == 0:
        return "All", True
    else:
        return "All", False
    
def reducerEuler(x, y):  # Test if all degress are even
    return x & y
    
for graph_file in graphs:
    graph = sc.textFile(graph_file)
    nodes = graph.flatMap(lambda line: line.split())
    node_degrees = nodes.map(lambda node: (node, 1)) 
    r1 = node_degrees.reduceByKey(lambda x,y: x + y)
    r2 = r1.map(map_func)
    print 'Has Euler tour? ', r2.reduceByKey(reducerEuler).collect()[0][1]



Has Euler tour?  True
Has Euler tour?  False
Has Euler tour?  True
Has Euler tour?  True
Has Euler tour?  False


In [165]:
import json
import pprint
# Exercise 7.3:
# You are given a couple of hours of raw WiFi data from my phone: 
# https://www.dropbox.com/s/964gq5o5bkzg7q3/wifi.data?dl=0
wifi_data = sc.textFile("../tests/wifi.data.txt")

# Compute the following things using Spark:
# 1. What are the 10 networks I observed the most, and how many times were they observed? 
# Note: the bssid is unique for every network, the name (ssid) of the network is not necessarily unique.

json_objects = wifi_data.map(lambda line: (eval(line)['bssid'], 1))
bssid_counts = json_objects.reduceByKey(lambda x,y: x + y)
pprint.pprint(bssid_counts.sortBy(lambda x:x[1], ascending=False).take(20))

[(u'34:21:09:12:6c:1a', 347),
 (u'00:24:b2:98:39:d2', 338),
 (u'34:21:09:12:6c:18', 324),
 (u'e8:08:8b:c9:c1:79', 318),
 (u'44:94:fc:56:08:fb', 315),
 (u'00:22:b0:b3:f2:ea', 314),
 (u'2c:b0:5d:ef:08:2b', 272),
 (u'44:94:fc:56:ce:5e', 240),
 (u'28:cf:e9:84:a1:c3', 211),
 (u'bc:ee:7b:55:1a:43', 210),
 (u'f8:1e:df:ff:a3:a8', 195),
 (u'bc:ee:7b:55:1a:42', 192),
 (u'28:cf:e9:84:a1:c2', 191),
 (u'b8:a3:86:50:cb:0c', 186),
 (u'e0:3f:49:ed:fc:e0', 176),
 (u'00:25:9c:3b:b1:72', 172),
 (u'84:1b:5e:df:5c:58', 161),
 (u'f8:1e:df:ff:a3:a7', 159),
 (u'c0:a0:bb:e8:da:c1', 139),
 (u'90:94:e4:83:ff:d6', 139)]


In [166]:
# 2. What are the 10 most common wifi names? (ssid)

json_objects = wifi_data.map(lambda line: (eval(line)['ssid'], 1))
ssid_counts = json_objects.reduceByKey(lambda x,y: x + y)
pprint.pprint(ssid_counts.sortBy(lambda x:x[1], ascending=False).take(10))

[(u'Kaspers Wi-Fi-netv\xe6rk', 402),
 (u'Internet4realz', 402),
 (u'AirLink5GHz126C18', 347),
 (u'NETGEAR_1', 338),
 (u'AirLink126C18', 324),
 (u'Housing People', 318),
 (u'Lausten_5GHz', 315),
 (u'Bronx', 314),
 (u'Playhouse', 272),
 (u'Lausten', 240)]


In [171]:
# 3. What are the 10 longest wifi names? (again, ssid)

json_objects = wifi_data.map(lambda line: (eval(line)['ssid'], 1))
unique_ssid = json_objects.reduceByKey(lambda x,y: x + y)
ssid_name_length = unique_ssid.map(lambda x: (x[0], len(x[0])))
pprint.pprint(ssid_name_length.sortBy(lambda x:x[1], ascending=False).take(20))

[(u'HP-Print-43-Deskjet 3520 series', 31),
 (u'TeliaGatewayA4-B1-E9-2C-9E-CA', 29),
 (u'TeliaGateway08-76-FF-84-FF-8C', 29),
 (u'TeliaGateway9C-97-26-57-15-F9', 29),
 (u'TeliaGateway08-76-FF-46-3E-36', 29),
 (u'TeliaGateway9C-97-26-57-15-99', 29),
 (u'TeliaGateway08-76-FF-8A-EE-32', 29),
 (u'TeliaGateway08-76-FF-85-04-2F', 29),
 (u'TeliaGateway08-76-FF-9C-E0-82', 29),
 (u'Charlotte R.s Wi-Fi-netv\xe6rk', 27),
 (u'Charlottes Wi-Fi-netv\xe6rk', 24),
 (u'Emil M\xf8rkebergs Netv\xe6rk', 23),
 (u'DIRECT-roku-562-F4B7AB', 22),
 (u'UnderKirkeMinisteriet', 21),
 (u'Kaspers Wi-Fi-netv\xe6rk', 21),
 (u'FC Midtjylland (5ghz)', 21),
 (u'Pia Lynnerups Netv\xe6rk', 21),
 (u'Apple Network 5e6d79', 20),
 (u'TLG18-Kongen-af-2100', 20),
 (u'Apple Network 20c5a7', 20)]
